In [1]:
# the purpose of this notebook is to explore the separated 10m, 30m, and 50m datasets and their aggregate data (average
# pitch/roll, variance in turbulence, battery loss, all that jazz)

In [2]:
# import tings
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
eg_log = pd.read_csv('FlightLogs/Jan9.12-44.csv')

In [31]:
# function that turns the files fed into it into a single row of aggregate data

# if file ends with _10m, entry in 'altitude' section is 10m, etc.
# include file name to check weird outliers out
# find average of all speeds, voltage, current, pitch, roll, battery_percent lost over time
# each time, append the row to the dataframe previously made

directory = Path('UsableLogs')
preaggregate_logs = directory.glob('*.csv')
output_dir = Path('AggregateLogs')

def process_usable_logs():
    aggregated_data = []

    for file_path in preaggregate_logs:        
        # Parse altitude from filename
        altitude = None
        if '_10m' in file_path.name:
            altitude = 10
        elif '_30m' in file_path.name:
            altitude = 30
        elif '_50m' in file_path.name:
            altitude = 50
        
        try:
            # Read the CSV
            df = pd.read_csv(file_path)
            start_time, end_time = pd.to_datetime(df['timestamp'].iloc[0]), pd.to_datetime(df['timestamp'].iloc[-1])
            flight_time = (end_time - start_time).total_seconds()
            
            # calculate the aggregate stats
            stats = {
                'file_name': file_path.stem,
                'flight_time(seconds)': flight_time,
                'altitude': altitude,
                'x_speed_avg': df['xSpeed(mph)'].mean(),
                'y_speed_avg': df['ySpeed(mph)'].mean(),
                'z_speed_avg': df['zSpeed(mph)'].mean(),
                'speed_avg': df['speed(mph)'].mean(),
                'speed_sd': df['speed(mph)'].std(),
                'voltage_avg': df['voltage(v)'].mean(),
                'voltage_sd': df['voltage(v)'].std(),
                'current_avg': df['current(A)'].mean(),
                'current_sd': df['current(A)'].std(),
                'pitch_avg': df['pitch(degrees)'].mean(),
                'pitch_sd': df['pitch(degrees)'].std(),
                'roll_avg': df['roll(degrees)'].mean(),
                'roll_sd': df['roll(degrees)'].std(),
                # We just take battery percent from first row minus last row
                'battery_loss_per_second': (df['battery_percent'].iloc[0] - df['battery_percent'].iloc[-1]) / flight_time
            }
            
            aggregated_data.append(stats)
            
        except Exception as e:
            print(f"Error processing {file_path.name}: {e}")

    # Return as a DataFrame
    return pd.DataFrame(aggregated_data)

In [32]:
process_usable_logs()

,file_name,flight_time(seconds),altitude,x_speed_avg,y_speed_avg,z_speed_avg,speed_avg,speed_sd,voltage_avg,voltage_sd,current_avg,current_sd,pitch_avg,pitch_sd,roll_avg,roll_sd,battery_loss_per_second
0,Jan11.3-29_50m,27.0,50,0.000000,0.000000,0.002523,0.000000,0.000000,7.712165,0.020490,4.831714,0.272000,6.200000,1.051145,-10.484586,1.214452,0.074074
1,Jan12.12-05_10m,25.0,10,0.009020,-0.002706,0.000000,0.023271,0.070071,7.834923,0.020025,4.616690,0.216424,3.878629,1.799164,-3.054839,1.056643,0.040000
2,Jan11.5-24_50m,24.0,50,0.000000,-0.008251,0.006417,0.010085,0.046508,7.354283,0.035355,5.220492,0.566170,12.233607,1.146410,-5.728279,1.505210,0.083333
3,Jan6.4-41_10m,16.0,10,0.000000,-0.002663,0.000000,0.002663,0.024334,7.348792,0.022183,5.007988,0.285532,4.276190,1.347530,-1.771429,0.676212,0.062500
4,Jan12.12-05_30m,24.0,30,-0.004680,0.005616,0.014039,0.024335,0.069798,7.788029,0.023261,4.673749,0.276785,4.051464,1.550274,-4.852720,0.871287,0.041667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,Jan12.12-11_50m,24.0,50,0.000000,0.000000,-0.005546,0.005546,0.034855,7.593909,0.018820,4.452017,0.214072,5.122314,1.159875,-2.861570,0.951070,0.083333
107,Jan11.5-20_10m,23.0,10,-0.003808,-0.002856,0.004759,0.022845,0.067883,7.707996,0.018890,4.606451,0.208286,3.270213,1.385412,-1.341702,1.092185,0.043478
108,Jan9.8-11_10m,24.0,10,0.000000,0.000000,0.000000,0.000000,0.000000,7.628640,0.007661,4.870606,0.193105,2.563559,0.610011,-0.694492,0.643603,0.041667
109,Jan7.3-35_10m,130.0,10,0.000733,-0.005134,0.074809,0.011735,0.049955,7.423626,0.041974,4.763856,0.376752,4.844262,1.272084,0.818361,1.462983,0.061538
